See reference: https://github.com/microsoft/Phi-3CookBook/blob/main/code/04.Finetuning/Phi-3-finetune-qlora-python.ipynb

He Zhang, Oct. 2024

## Installing and loading the libraries

In [1]:
#%pip install --upgrade bitsandbytes
#%pip install --upgrade transformers
#%pip install --upgrade peft 
#%pip install --upgrade accelerate 
#%pip install --upgrade datasets 
#%pip install --upgrade trl 
#%pip install --upgrade flash_attn 
#%pip install --upgrade torch 
#%pip install --upgrade wandb

#%pip install huggingface_hub
#%pip install python-dotenv
#%pip install ipywidgets

#%pip install --upgrade absl-py 
#%pip install --upgrade nltk 
#%pip install --upgrade rouge_score
#%pip install --upgrade evaluate

## Importing the libraries

In [2]:
import torch

from random import randrange

from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)

from trl import SFTTrainer

## Setting Global Parameters

In [3]:
# 'model_id' and 'model_name' are the identifiers for the pre-trained model that you want to fine-tune. 
# In this case, it's the 'Phi-3-mini-4k-instruct' model from Microsoft.
# microsoft/Phi-3-mini-4k-instruct
# microsoft/Phi-3-small-128k-instruct
model_id = "microsoft/Phi-3-mini-4k-instruct"

model_name = "microsoft/Phi-3-mini-4k-instruct"

# In this case, it's the 'python_code_instructions_18k_alpaca' dataset from iamtarun (Ex: iamtarun/python_code_instructions_18k_alpaca).
# Update Dataset Name to your dataset name
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"

# 'dataset_split' is the split of the dataset that you want to use for training. 
# In this case, it's the 'train' split.
dataset_split= "train"

# 'new_model' is the name that you want to give to the fine-tuned model.
new_model = "phi-3-mini-4k-qlora-ft-code-gen"

# 'hf_model_repo' is the repository on the Hugging Face Model Hub where the fine-tuned model will be saved. Update UserName to your Hugging Face Username
hf_model_repo="HeZ/"+new_model

# 'device_map' is a dictionary that maps the model to the GPU device. 
# In this case, the entire model is loaded on GPU 0.
device_map = {"": 0}

# Bits and Bytes configuration for the model

# 'use_4bit' is a boolean that controls whether 4-bit precision should be used for loading the base model.
use_4bit = True

# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True

# The following are parameters for the LoRA (Learning from Random Architecture) model.

# 'lora_r' is the dimension of the LoRA attention.
lora_r = 16

# 'lora_alpha' is the alpha parameter for LoRA scaling.
lora_alpha = 16

# 'lora_dropout' is the dropout probability for LoRA layers.
lora_dropout = 0.05

# 'target_modules' is a list of the modules in the model that will be replaced with LoRA layers.
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]

# 'set_seed' is a function that sets the seed for generating random numbers, 
# which is used for reproducibility of the results.
set_seed(1234)

## Connect to Huggingface Hub

In [4]:
# This code block is used to log in to the Hugging Face Model Hub using an API token stored in an environment variable.

# 'login' is a function from the 'huggingface_hub' library that logs you in to the Hugging Face Model Hub using an API token.
from huggingface_hub import login, model_info

# 'load_dotenv' is a function from the 'python-dotenv' library that loads environment variables from a .env file.
from dotenv import load_dotenv

# 'os' is a standard Python library that provides functions for interacting with the operating system.
import os

# Call the 'load_dotenv' function to load the environment variables from the .env file.
load_dotenv("hf_token.env")

# Call the 'login' function with the 'HF_HUB_TOKEN' environment variable to log in to the Hugging Face Model Hub.
# 'os.getenv' is a function that gets the value of an environment variable.
login(token=os.getenv("HF_HUB_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/azureuser/.cache/huggingface/token
Login successful


In [5]:
# login check to see some model information
test_model_id = "HeZ/test-repo-v1" 
info = model_info(test_model_id)
print(info)

ModelInfo(id='HeZ/test-repo-v1', author='HeZ', sha='379f90c9789e65163946e8b0518641e365fcbfec', created_at=datetime.datetime(2024, 10, 1, 8, 34, 7, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 10, 1, 8, 34, 7, tzinfo=datetime.timezone.utc), private=False, disabled=False, downloads=0, downloads_all_time=None, gated=False, gguf=None, inference=None, likes=0, library_name=None, tags=['region:us'], pipeline_tag=None, mask_token=None, card_data=None, widget_data=None, model_index=None, config=None, transformers_info=None, trending_score=None, siblings=[RepoSibling(rfilename='.gitattributes', size=None, blob_id=None, lfs=None)], spaces=[], safetensors=None)


## Load the dataset with the instruction set

In [6]:
# This code block is used to load a dataset from the Hugging Face Dataset Hub, print its size, and show a random example from the dataset.

# 'load_dataset' is a function from the 'datasets' library that loads a dataset from the Hugging Face Dataset Hub.
# 'dataset_name' is the name of the dataset to load, and 'dataset_split' is the split of the dataset to load (e.g., 'train', 'test').
dataset = load_dataset(dataset_name, split=dataset_split)

dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 18612
})

In [7]:
# The 'len' function is used to get the size of the dataset, which is then printed.
print(f"dataset size: {len(dataset)}")

display(dataset[0])
print(dataset[0]["prompt"])

dataset size: 18612


{'instruction': 'Create a function to calculate the sum of a sequence of integers.',
 'input': '[1, 2, 3, 4, 5]',
 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum'}

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a function to calculate the sum of a sequence of integers.

### Input:
[1, 2, 3, 4, 5]

### Output:
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum


## Load the tokenizer to prepare the dataset

In [8]:
# 'tokenizer_id' is the identifier for the tokenizer that you want to load. In this case, it is set to the value of 'model_id', which means that the tokenizer associated with the pre-trained model will be loaded.

# 'AutoTokenizer' is a class from the 'transformers' library that provides a generic tokenizer class from which all other tokenizer classes inherit.

# 'from_pretrained' is a method of the 'AutoTokenizer' class that loads a tokenizer from the Hugging Face Model Hub.

# 'tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)' loads the tokenizer associated with 'tokenizer_id' from the Hugging Face Model Hub and assigns it to the variable 'tokenizer'.

# 'tokenizer.padding_side' is a property of the 'tokenizer' object that determines on which side of the input sequences padding should be added. It can be set to either 'left' or 'right'.

# 'tokenizer.padding_side = 'right'' sets 'tokenizer.padding_side' to 'right', which means that padding will be added to the right side of the input sequences. This is done to prevent warnings that can occur when 'tokenizer.padding_side' is set to 'left'.
tokenizer_id = model_id
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right' # to prevent warnings

In [9]:
# 'create_message_column' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'messages' and the value is a list of dictionaries, each representing a message.

# 'row' is the input to the 'create_message_column' function. It is expected to be a dictionary with keys 'instruction', 'input', and 'output'.

# 'messages' is a list that will contain the messages.

# 'user' is a dictionary that represents a user message. The 'content' key contains the instruction and input from the row, and the 'role' key is set to 'user'.

# 'messages.append(user)' adds the user message to the 'messages' list.

# 'assistant' is a dictionary that represents an assistant message. The 'content' key contains the output from the row, and the 'role' key is set to 'assistant'.

# 'messages.append(assistant)' adds the assistant message to the 'messages' list.

# 'return {"messages": messages}' returns a dictionary with a single key-value pair. The key is 'messages' and the value is the 'messages' list.

# 'format_dataset_chatml' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row.

# 'row' is the input to the 'format_dataset_chatml' function. It is expected to be a dictionary with a key 'messages'.

# 'return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}' returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row. The 'add_generation_prompt' parameter is set to False, which means that no generation prompt will be added to the end of the text. The 'tokenize' parameter is set to False, which means that the text will not be tokenized.
def create_message_column(row):
    messages = []
    user = {
        "content": f"{row['instruction']}\n Input: {row['input']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

In [10]:
# This code block is used to prepare the 'dataset' for training a chat model.

# 'dataset.map' is a method that applies a function to each example in the 'dataset'.
# 'create_message_column' is a function that formats each example into a 'messages' format suitable for a chat model.
# The result is a new 'dataset_chatml' with the formatted examples.
dataset_chatml = dataset.map(create_message_column)

# 'dataset_chatml.map' is a method that applies a function to each example in the 'dataset_chatml'.
# 'format_dataset_chatml' is a function that further formats each example into a single string of chat messages.
# The result is an updated 'dataset_chatml' with the further formatted examples.
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

In [11]:
# This line of code is used to access and display the first example from the 'dataset_chatml'.

# 'dataset_chatml[0]' uses indexing to access the first example in the 'dataset_chatml'.
# In Python, indexing starts at 0, so 'dataset_chatml[0]' refers to the first example.
# The result is a dictionary with a 'text' key and a string of formatted chat messages as its value.
display(dataset_chatml)

display(dataset_chatml[0])

print(dataset_chatml[0]["text"])

Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
    num_rows: 18612
})

{'instruction': 'Create a function to calculate the sum of a sequence of integers.',
 'input': '[1, 2, 3, 4, 5]',
 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
 'messages': [{'content': 'Create a function to calculate the sum of a sequence of integers.\n Input: [1, 2, 3, 4, 5]',
   'role': 'user'},
  {'content': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum',
   'role': 'assistant'}],
 'text': '<|user|>\nCreate a function to calculate the sum of a sequence of integers.\n Input: [1, 2, 3, 4, 5]<

<|user|>
Create a function to calculate the sum of a sequence of integers.
 Input: [1, 2, 3, 4, 5]<|end|>
<|assistant|>
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum<|end|>
<|endoftext|>


In [12]:
# This code block is used to split the 'dataset_chatml' into training and testing sets.

# 'dataset_chatml.train_test_split' is a method that splits the 'dataset_chatml' into a training set and a testing set.
# 'test_size' is a parameter that specifies the proportion of the 'dataset_chatml' to include in the testing set. Here it's set to 0.05, meaning that 5% of the 'dataset_chatml' will be included in the testing set.
# 'seed' is a parameter that sets the seed for the random number generator. This is used to ensure that the split is reproducible. Here it's set to 1234.
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)

# This line of code is used to display the structure of the 'dataset_chatml' after the split.
# It will typically show information such as the number of rows in the training set and the testing set.
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 17681
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 931
    })
})

## Instruction fine-tune a Phi-3-mini model using QLORA and trl

In [13]:
# This code block is used to set the compute data type and attention implementation based on whether bfloat16 is supported on the current CUDA device.

# 'torch.cuda.is_bf16_supported()' is a function that checks if bfloat16 is supported on the current CUDA device.
# If bfloat16 is supported, 'compute_dtype' is set to 'torch.bfloat16' and 'attn_implementation' is set to 'flash_attention_2'.
if torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
    attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
    compute_dtype = torch.float16
    attn_implementation = 'sdpa'

# This line of code is used to print the value of 'attn_implementation', which indicates the chosen attention implementation.
print(attn_implementation)

flash_attention_2


In [14]:
compute_dtype

torch.bfloat16

In [15]:
# set flash attention to be 'eager' mode, if you run this notebook using Nvidia V100 GPU
attn_implementation = 'eager'

## Load the tokenizer and model to finetune

In [16]:
# 'AutoTokenizer' is a class from the Hugging Face Transformers library that provides a tokenizer for a given pre-trained model.

# 'from_pretrained' is a method of the 'AutoTokenizer' class that loads a tokenizer from a pre-trained model.

# 'model_name' is a variable that contains the name of the pre-trained model.

# 'trust_remote_code=True' is a parameter that allows the execution of remote code when loading the tokenizer.

# 'add_eos_token=True' is a parameter that adds an end-of-sentence token to the tokenizer.

# 'use_fast=True' is a parameter that uses the fast version of the tokenizer, if available.

# 'tokenizer.pad_token = tokenizer.unk_token' sets the padding token of the tokenizer to be the same as the unknown token.

# 'tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)' sets the ID of the padding token to be the same as the ID of the padding token.

# 'tokenizer.padding_side = 'left'' sets the side where padding will be added to be the left side.

# 'BitsAndBytesConfig' is a class that provides a configuration for quantization.

# 'bnb_config' is a variable that holds the configuration for quantization.

# 'AutoModelForCausalLM' is a class from the Hugging Face Transformers library that provides a model for causal language modeling.

# 'from_pretrained' is a method of the 'AutoModelForCausalLM' class that loads a model from a pre-trained model.

# 'torch_dtype=compute_dtype' is a parameter that sets the data type of the model to be the same as 'compute_dtype'.

# 'quantization_config=bnb_config' is a parameter that sets the configuration for quantization to be 'bnb_config'.

# 'device_map=device_map' is a parameter that sets the device map of the model to be 'device_map'.

# 'attn_implementation=attn_implementation' is a parameter that sets the type of attention implementation to be 'attn_implementation'.

# 'prepare_model_for_kbit_training' is a function that prepares a model for k-bit training.

# 'model = prepare_model_for_kbit_training(model)' prepares 'model' for k-bit training and assigns the result back to 'model'.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_double_quant,
)

model = AutoModelForCausalLM.from_pretrained(
          model_name, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map=device_map,
          attn_implementation=attn_implementation
)

model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
# This code block is used to define the training arguments for the model.

# 'TrainingArguments' is a class that holds the arguments for training a model.
# 'output_dir' is the directory where the model and its checkpoints will be saved.
# 'evaluation_strategy' is set to "steps", meaning that evaluation will be performed after a certain number of training steps.
# 'do_eval' is set to True, meaning that evaluation will be performed.
# 'optim' is set to "adamw_torch", meaning that the AdamW optimizer from PyTorch will be used.
# 'per_device_train_batch_size' and 'per_device_eval_batch_size' are set to 8, meaning that the batch size for training and evaluation will be 8 per device.
# 'gradient_accumulation_steps' is set to 4, meaning that gradients will be accumulated over 4 steps before performing a backward/update pass.
# 'log_level' is set to "debug", meaning that all log messages will be printed.
# 'save_strategy' is set to "epoch", meaning that the model will be saved after each epoch.
# 'logging_steps' is set to 100, meaning that log messages will be printed every 100 steps.
# 'learning_rate' is set to 1e-4, which is the learning rate for the optimizer.
# 'fp16' is set to the opposite of whether bfloat16 is supported on the current CUDA device.
# 'bf16' is set to whether bfloat16 is supported on the current CUDA device.
# 'eval_steps' is set to 100, meaning that evaluation will be performed every 100 steps.
# 'num_train_epochs' is set to 3, meaning that the model will be trained for 3 epochs.
# 'warmup_ratio' is set to 0.1, meaning that 10% of the total training steps will be used for the warmup phase.
# 'lr_scheduler_type' is set to "linear", meaning that a linear learning rate scheduler will be used.
# 'report_to' is set to "wandb", meaning that training and evaluation metrics will be reported to Weights & Biases.
# 'seed' is set to 42, which is the seed for the random number generator.

# LoraConfig object is created with the following parameters:
# 'r' (rank of the low-rank approximation) is set to 16,
# 'lora_alpha' (scaling factor) is set to 16,
# 'lora_dropout' dropout probability for Lora layers is set to 0.05,
# 'task_type' (set to TaskType.CAUSAL_LM indicating the task type),
# 'target_modules' (the modules to which LoRA is applied) choosing linear layers except the output layer..

args = TrainingArguments(
        output_dir="./phi-3-mini-QLoRA",
        eval_strategy="steps",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=10,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=10,
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        report_to="wandb",
        seed=42,
)

peft_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)

## Establish a connection with wandb and enlist the project and experiment.

In [18]:
# This code block is used to initialize Weights & Biases (wandb), a tool for tracking and visualizing machine learning experiments.

# 'import wandb' is used to import the wandb library.
import wandb

# 'wandb.login()' is a method that logs you into your Weights & Biases account.
# If you're not already logged in, it will prompt you to log in.
# Once you're logged in, you can use Weights & Biases to track and visualize your experiments.
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hezhang33. Use `wandb login --relogin` to force relogin


True

In [19]:
# This code block is used to initialize a Weights & Biases (wandb) run.

# 'project_name' is set to the name of the project in Weights & Biases.
project_name = "Phi3-mini-qlora-ft-python-code"

# 'wandb.init' is a method that initializes a new Weights & Biases run.
# 'project' is set to 'project_name', meaning that the run will be associated with this project.
# 'name' is set to "phi-3-mini-ft-py-3e", which is the name of the run.
# Each run has a unique name which can be used to identify it in the Weights & Biases dashboard.
wandb.init(project=project_name, name="phi-3-mini-qlora-ft-py-3e-runs")

In [20]:
# 'SFTTrainer' is a class that provides a trainer for fine-tuning a model.

# 'trainer' is a variable that holds the trainer.

# 'model=model' is a parameter that sets the model to be trained to be 'model'.

# 'train_dataset=dataset_chatml['train']' is a parameter that sets the training dataset to be 'dataset_chatml['train']'.

# 'eval_dataset=dataset_chatml['test']' is a parameter that sets the evaluation dataset to be 'dataset_chatml['test']'.

# 'peft_config=peft_config' is a parameter that sets the configuration for the Lora layer to be 'peft_config'.

# 'dataset_text_field="text"' is a parameter that sets the field in the dataset that contains the text to be 'text'.

# 'max_seq_length=512' is a parameter that sets the maximum sequence length for the model to be 512.

# 'tokenizer=tokenizer' is a parameter that sets the tokenizer to be 'tokenizer'.

# 'args=args' is a parameter that sets the training arguments to be 'args'.

# This line of code is used to create a trainer for fine-tuning the model with the specified parameters.
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml['train'],
        eval_dataset=dataset_chatml['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=args
)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Using auto half precision backend


#### Note by He Zhang: total number of training steps = (total samples * number of epochs) / gradient accumulation steps

In [21]:
%%time
# This code block is used to train the model and save it locally.

# 'trainer.train()' is a method that starts the training of the model.
# It uses the training dataset, evaluation dataset, and training arguments that were provided when the trainer was initialized.
trainer.train()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 17,681
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 552
  Number of trainable parameters = 8,912,896
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not

Step,Training Loss,Validation Loss
10,1.019300,1.058206
20,1.040800,1.012917
30,0.995700,0.909531
40,0.804700,0.771116
50,0.699000,0.668948
60,0.614300,0.638497
70,0.647200,0.617485
80,0.607700,0.607318
90,0.602800,0.601198
100,0.592900,0.597763



***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evaluation *****
  Num examples = 931
  Batch size = 8

***** Running Evalu

In [30]:
hf_model_repo

'HeZ/phi-3-mini-4k-qlora-ft-code-gen'

In [24]:
# 'trainer.save_model()' is a method that saves the trained model locally.
# The model will be saved in the directory specified by 'output_dir' in the training arguments.
trainer.save_model()

Saving model checkpoint to ./phi-3-mini-QLoRA
loading configuration file config.json from cache at /home/azureuser/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000

In [25]:
# This code block is used to save the adapter to the Hugging Face Model Hub.

# 'trainer.push_to_hub' is a method that pushes the trained model (or adapter in this case) to the Hugging Face Model Hub.
# In this example, hf_model_repo="HeZ/phi-3-mini-4k-ft-code-gen", i.e. the name of the repository on the Hugging Face Model Hub where the adapter will be saved.
trainer.push_to_hub(hf_model_repo)

Saving model checkpoint to ./phi-3-mini-QLoRA
loading configuration file config.json from cache at /home/azureuser/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/HeZ/phi-3-mini-QLoRA/commit/eef1a66c9c56efa74218208302f444a74e24f006', commit_message='HeZ/phi-3-mini-4k-qlora-ft-code-gen', commit_description='', oid='eef1a66c9c56efa74218208302f444a74e24f006', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HeZ/phi-3-mini-QLoRA', endpoint='https://huggingface.co', repo_type='model', repo_id='HeZ/phi-3-mini-QLoRA'), pr_revision=None, pr_num=None)

## Combine the model and the adapters, then save it.

In [27]:
# 'del model' and 'del trainer' are lines of code that delete the 'model' and 'trainer' objects. This frees up the memory that was used by these objects.

# 'import gc' is a line of code that imports the 'gc' module, which provides an interface to the garbage collector.

# 'gc.collect()' is a function that triggers a full garbage collection. It frees up memory by collecting all the objects that are no longer in use.

# This block of code is used to empty the VRAM (Video Random Access Memory) by deleting the 'model' and 'trainer' objects and then triggering a full garbage collection.
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

0

In [28]:
# 'torch.cuda.empty_cache()' is a function from the PyTorch library that releases all unoccupied cached memory currently held by the caching allocator so that those can be used in other GPU application and visible in nvidia-smi.

# It's a PyTorch specific function to manage GPU memory and it doesn't affect the GPU memory usage by PyTorch tensors.

# This line of code is used to empty the cache memory that's used by PyTorch on the GPU.
torch.cuda.empty_cache() # PyTorch thing
torch.cuda.empty_cache() # PyTorch thing

In [32]:
# 'hf_adapter_repo' is a variable that holds the repository name for the Hugging Face model adapter.

# 'edumunozsala/phi-3-mini-QLoRA' is the repository name, where 'edumunozsala' is the username of the repository owner and 'phi-3-mini-QLoRA' is the name of the model adapter.

# 'model_name, hf_adapter_repo, compute_dtype' is a line of code that returns the values of the 'model_name', 'hf_adapter_repo', and 'compute_dtype' variables.

# This block of code is used to set the repository name for the Hugging Face model adapter and then return the values of the 'model_name', 'hf_adapter_repo', and 'compute_dtype' variables.
hf_adapter_repo = "HeZ/phi-3-mini-QLoRA"

model_name, hf_adapter_repo, compute_dtype

('microsoft/Phi-3-mini-4k-instruct', 'HeZ/phi-3-mini-QLoRA', torch.bfloat16)

In [33]:
# 'peft_model_id' and 'tr_model_id' are variables that hold the identifiers for the PEFT model and the transformer model, respectively.

# 'AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype=compute_dtype)' is a function that loads a pre-trained transformer model for causal language modeling. 'tr_model_id' is the identifier for the pre-trained model, 'trust_remote_code=True' allows the execution of code from the model file, and 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors.

# 'PeftModel.from_pretrained(model, peft_model_id)' is a function that loads a pre-trained PEFT model. 'model' is the transformer model and 'peft_model_id' is the identifier for the pre-trained PEFT model.

# 'model.merge_and_unload()' is a method that merges the PEFT model with the transformer model and then unloads the PEFT model.

# This block of code is used to load a pre-trained transformer model and a pre-trained PEFT model, merge the two models, and then unload the PEFT model.
peft_model_id = hf_adapter_repo
tr_model_id = model_name

model = AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype=compute_dtype)
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.merge_and_unload()

loading configuration file config.json from cache at /home/azureuser/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
loading configuration file config.json from cache at /home/azureuser/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type":

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/azureuser/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



adapter_config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

In [34]:
# 'tokenizer' is a variable that holds the tokenizer.

# 'AutoTokenizer.from_pretrained(peft_model_id)' is a function from the Hugging Face Transformers library that loads a pre-trained tokenizer. 'peft_model_id' is the identifier for the pre-trained tokenizer.

# This line of code is used to load a pre-trained tokenizer.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

tokenizer_config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-QLoRA/snapshots/eef1a66c9c56efa74218208302f444a74e24f006/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-QLoRA/snapshots/eef1a66c9c56efa74218208302f444a74e24f006/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-QLoRA/snapshots/eef1a66c9c56efa74218208302f444a74e24f006/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
# 'hf_model_repo' is a variable that holds the repository name for the Hugging Face model.

# This line of code is used to reference the repository name for the Hugging Face model.
hf_model_repo

'HeZ/phi-3-mini-4k-qlora-ft-code-gen'

In [36]:
# 'merged_model_id' is a variable that holds the identifier for the merged model.

# 'hf_model_repo' is the repository name for the Hugging Face model.

# 'model.push_to_hub(merged_model_id)' is a method that pushes the merged model to the Hugging Face Model Hub. 'merged_model_id' is the identifier for the merged model.

# 'tokenizer.push_to_hub(merged_model_id)' is a method that pushes the tokenizer to the Hugging Face Model Hub. 'merged_model_id' is the identifier for the tokenizer.

# This block of code is used to save the merged model and the tokenizer to the Hugging Face Model Hub.
# SAve the model merged to the Hub
merged_model_id = hf_model_repo
model.push_to_hub(merged_model_id)
tokenizer.push_to_hub(merged_model_id)

Configuration saved in /tmp/tmpkki8w97i/config.json
Configuration saved in /tmp/tmpkki8w97i/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at /tmp/tmpkki8w97i/model.safetensors.index.json.
Uploading the following files to HeZ/phi-3-mini-4k-qlora-ft-code-gen: model-00002-of-00002.safetensors,config.json,README.md,generation_config.json,model-00001-of-00002.safetensors,model.safetensors.index.json


model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmpkte16ssg/tokenizer_config.json
Special tokens file saved in /tmp/tmpkte16ssg/special_tokens_map.json
Uploading the following files to HeZ/phi-3-mini-4k-qlora-ft-code-gen: README.md,special_tokens_map.json,tokenizer.json,tokenizer_config.json


CommitInfo(commit_url='https://huggingface.co/HeZ/phi-3-mini-4k-qlora-ft-code-gen/commit/b4dd44f929efe05ce90b6ec9e368c6ddc0e1941c', commit_message='Upload tokenizer', commit_description='', oid='b4dd44f929efe05ce90b6ec9e368c6ddc0e1941c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HeZ/phi-3-mini-4k-qlora-ft-code-gen', endpoint='https://huggingface.co', repo_type='model', repo_id='HeZ/phi-3-mini-4k-qlora-ft-code-gen'), pr_revision=None, pr_num=None)

## Model Inference and evaluation

In [37]:
# 'hf_model_repo' is a variable that holds the name of the repository on the Hugging Face Model Hub.
# This is where the trained and merged model, as well as the tokenizer, have been saved.
hf_model_repo

'HeZ/phi-3-mini-4k-qlora-ft-code-gen'

In [38]:
# 'device_map' is a variable that holds the mapping of the devices that are used for computation.

# 'compute_dtype' is a variable that holds the data type that is used for computation.

# This line of code is used to return the values of the 'device_map' and 'compute_dtype' variables.
device_map, compute_dtype

({'': 0}, torch.bfloat16)

In [39]:
# This block of code is used to import the necessary libraries, set the seed for reproducibility, and load a pre-trained tokenizer and model.

# 'import torch' is a line of code that imports the PyTorch library, which is a popular open-source machine learning library.

# 'from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed' is a line of code that imports the 'AutoTokenizer', 'AutoModelForCausalLM', and 'set_seed' functions from the Hugging Face Transformers library.

# 'set_seed(1234)' is a line of code that sets the seed for the random number generator to '1234'. This is done to ensure that the results are reproducible.

# 'tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)' is a line of code that loads a pre-trained tokenizer from the Hugging Face Model Hub. 'hf_model_repo' is the repository name for the model and 'trust_remote_code=True' allows the execution of code from the model file.

# 'model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype=compute_dtype, device_map=device_map)' is a line of code that loads a pre-trained model for causal language modeling from the Hugging Face Model Hub. 'hf_model_repo' is the repository name for the model, 'trust_remote_code=True' allows the execution of code from the model file, 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors, and 'device_map=device_map' sets the device mapping.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(1234)  # For reproducibility

tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype=compute_dtype, device_map=device_map) # compute "auto" dev_map "cuda"

tokenizer_config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-4k-qlora-ft-code-gen/snapshots/b4dd44f929efe05ce90b6ec9e368c6ddc0e1941c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-4k-qlora-ft-code-gen/snapshots/b4dd44f929efe05ce90b6ec9e368c6ddc0e1941c/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-4k-qlora-ft-code-gen/snapshots/b4dd44f929efe05ce90b6ec9e368c6ddc0e1941c/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

...

Instantiating Phi3ForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "pad_token_id": 32000
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at HeZ/phi-3-mini-4k-qlora-ft-code-gen.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /home/azureuser/.cache/huggingface/hub/models--HeZ--phi-3-mini-4k-qlora-ft-code-gen/snapshots/b4dd44f929efe05ce90b6ec9e368c6ddc0e1941c/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



#### Inference without pre-processing

In [40]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 To solve the equation 2x + 3 = 7, you need to isolate the variable x. First, subtract 3 from both sides of the equation to get 2x = 4. Then, divide both sides by 2 to get x = 2. So the solution to the equation is x = 2.


#### Inference with pre-processing

In [42]:
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 17681
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text'],
        num_rows: 931
    })
})

In [43]:
# 'dataset_chatml['test'][0]' is used to access the first element of the test set in the 'dataset_chatml' dataset.
# This can be used to inspect the first test sample to understand its structure and contents.
dataset_chatml['test'][0]

{'instruction': 'Create an algorithm in Python to sort an array of numbers.',
 'input': '[9, 3, 5, 1, 6]',
 'output': 'def sortArray(arr):\n    for i in range(len(arr)):\n        for j in range(i+1,len(arr)):\n            if arr[i] > arr[j]:\n                temp = arr[i]\n                arr[i] = arr[j]\n                arr[j] = temp\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sortArray(arr))',
 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate an algorithm in Python to sort an array of numbers.\n\n### Input:\n[9, 3, 5, 1, 6]\n\n### Output:\ndef sortArray(arr):\n    for i in range(len(arr)):\n        for j in range(i+1,len(arr)):\n            if arr[i] > arr[j]:\n                temp = arr[i]\n                arr[i] = arr[j]\n                arr[j] = temp\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sortArray(arr))',
 'messages': [{'content': 'Create an algorithm in Python to sort an 

In [44]:
# 'pipeline' is a function from the 'transformers' library that creates a pipeline for text generation.
# 'text-generation' is the task that the pipeline will perform.
# 'model' is the pre-trained model that the pipeline will use.
# 'tokenizer' is the tokenizer that the pipeline will use to tokenize the input text.
# The created pipeline is stored in the 'pipe' variable.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [45]:
# This code block defines a function 'test_inference' that performs inference on a given prompt.

# 'prompt' is the input to the function. It is the text that the model will generate a response to.

# 'pipe.tokenizer.apply_chat_template' is a method that applies the chat template to the prompt.
# The prompt is wrapped in a list and formatted as a dictionary with "role" set to "user" and "content" set to the prompt.
# 'tokenize' is set to False, which means that the method will not tokenize the prompt.
# 'add_generation_prompt' is set to True, which means that the method will add a generation prompt to the prompt.
# The formatted prompt is stored back in the 'prompt' variable.

# 'pipe' is the text generation pipeline that was created earlier.
# It is called with the formatted prompt and several parameters that control the text generation process.
# 'max_new_tokens=256' limits the maximum number of new tokens that can be generated.
# 'do_sample=True' enables sampling, which means that the model will generate diverse responses.
# 'num_beams=1' sets the number of beams for beam search to 1, which means that the model will generate one response.
# 'temperature=0.3' controls the randomness of the responses. Lower values make the responses more deterministic.
# 'top_k=50' limits the number of highest probability vocabulary tokens to consider for each step.
# 'top_p=0.95' enables nucleus sampling and sets the cumulative probability of parameter tokens to 0.95.
# 'max_time=180' limits the maximum time for the generation process to 180 seconds.
# The generated responses are stored in the 'outputs' variable.

# The function returns the first generated response.
# The response is stripped of the prompt and any leading or trailing whitespace.


# Original version from the notebook - not able to call model pipeline
"""
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95,
                   max_time= 180) #, eos_token_id=eos_token)
    return outputs[0]['generated_text'][len(prompt):].strip()
"""

# Modified version by He Zhang Oct. 2024 - able to call model pipeline with success!
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe([{"role":"user", "content":prompt}], max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95,
                   max_time= 180) #, eos_token_id=eos_token)
    return outputs[0]['generated_text'][1]["content"].strip()

In [46]:
# This code block calls the 'test_inference' function with the first message in the test set of 'dataset_chatml' as the prompt.
# 'test_inference' performs inference on the prompt and returns a generated response.
# The response is printed to the console.
print(test_inference(dataset_chatml['test'][0]['messages'][0]['content']))

def sort_array(arr):
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[min_idx] > arr[j]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

arr = [9, 3, 5, 1, 6]
print(sort_array(arr))


In [47]:
# He Zhang Oct. 2024: testing the phi3 generated response - works correctly
def sort_array(arr):
    for i in range(len(arr)):
        min_idx = i
        for j in range(i+1, len(arr)):
            if arr[min_idx] > arr[j]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

arr = [9, 3, 5, 1, 6]
print(sort_array(arr))

[1, 3, 5, 6, 9]


## Evaluate the performance

In [48]:
#%pip install --upgrade evaluate

In [49]:
# 'load_metric' is a function from the 'datasets' library that loads a metric for evaluating the model.
# Metrics are used to measure the performance of the model on certain tasks.

#from datasets import load_metric # note by He Zhang, Oct. 2024: after datasets version 3.0.0, there is no load_metric() function any more!

import evaluate

rouge_metric = evaluate.load('rouge')

In [50]:
# test the rouge score
rouge_metric.compute(predictions=["hi, my name is He Zhang"], 
                     references=["hi, i am a boy, and my name is He Zhang."], 
                     use_stemmer=True)

{'rouge1': 0.7058823529411764,
 'rouge2': 0.5333333333333333,
 'rougeL': 0.7058823529411764,
 'rougeLsum': 0.7058823529411764}

In [51]:
# This code block defines a function 'calculate_rogue' that calculates the ROUGE score for a given row in the dataset.

# 'row' is the input to the function. It is a row in the dataset that contains a message and its corresponding output.

# 'test_inference(row['messages'][0]['content'])' calls the 'test_inference' function with the first message in the row as the prompt.
# 'test_inference' performs inference on the prompt and returns a generated response.
# The response is stored in the 'response' variable.

# 'rouge_metric.compute' is a method that calculates the ROUGE score for the generated response and the corresponding output in the row.
# 'predictions' is set to the generated response and 'references' is set to the output in the row.
# 'use_stemmer' is set to True, which means that the method will use a stemmer to reduce words to their root form.
# The calculated ROUGE score is stored in the 'result' variable.

# The 'result' dictionary is updated to contain the F-measure of each ROUGE score multiplied by 100.
# The F-measure is a measure of a test's accuracy that considers both the precision and the recall of the test.

# The 'response' is added to the 'result' dictionary.

# The function returns the 'result' dictionary.
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    #result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [52]:
%%time
# '%%time' is a magic command in Jupyter notebooks that measures the execution time of the cell.

# 'dataset_chatml['test'].select(range(0,500))' selects the first 500 elements from the test set in the 'dataset_chatml' dataset.

# '.map(calculate_rogue, batched=False)' applies the 'calculate_rogue' function to each element in the selected subset.
# 'calculate_rogue' calculates the ROUGE score for each element.
# 'batched' is set to False, which means that the function will be applied to each element individually, not in batches.

# The results are stored in the 'metricas' variable.
metricas = dataset_chatml['test'].select(range(0, 10)).map(calculate_rogue, batched=False)

Parameter 'function'=<function calculate_rogue at 0x7fe460764160> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


CPU times: user 1min 7s, sys: 325 ms, total: 1min 7s
Wall time: 1min 7s


In [53]:
metricas

Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'messages', 'text', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum', 'response'],
    num_rows: 10
})

In [54]:
metricas[0]

{'instruction': 'Create an algorithm in Python to sort an array of numbers.',
 'input': '[9, 3, 5, 1, 6]',
 'output': 'def sortArray(arr):\n    for i in range(len(arr)):\n        for j in range(i+1,len(arr)):\n            if arr[i] > arr[j]:\n                temp = arr[i]\n                arr[i] = arr[j]\n                arr[j] = temp\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sortArray(arr))',
 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate an algorithm in Python to sort an array of numbers.\n\n### Input:\n[9, 3, 5, 1, 6]\n\n### Output:\ndef sortArray(arr):\n    for i in range(len(arr)):\n        for j in range(i+1,len(arr)):\n            if arr[i] > arr[j]:\n                temp = arr[i]\n                arr[i] = arr[j]\n                arr[j] = temp\n    return arr\n\narr = [9, 3, 5, 1, 6]\nprint(sortArray(arr))',
 'messages': [{'content': 'Create an algorithm in Python to sort an 

In [55]:
import numpy as np

In [56]:
# This code block prints the mean of the ROUGE-1, ROUGE-2, ROUGE-L, and ROUGE-Lsum scores in the 'metricas' dictionary.

# 'np.mean(metricas['rouge1'])' calculates the mean of the ROUGE-1 scores.
# 'np.mean(metricas['rouge2'])' calculates the mean of the ROUGE-2 scores.
# 'np.mean(metricas['rougeL'])' calculates the mean of the ROUGE-L scores.
# 'np.mean(metricas['rougeLsum'])' calculates the mean of the ROUGE-Lsum scores.

# 'print' is used to print the calculated means to the console.
print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))
print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))
print("Rouge L Mean: ",np.mean(metricas['rougeL']))
print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))

Rouge 1 Mean:  0.5792650524219705
Rouge 2 Mean:  0.3335120724777107
Rouge L Mean:  0.5004273854576352
Rouge Lsum Mean:  0.5776902492723642


In [63]:
# check model size
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 7288.137MB
